<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Вывод</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
try:
    import pandas_profiling
except:
    !pip install pandas-profiling
    import pandas_profiling

In [2]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

In [3]:
np.random.seed(sum(ord(x) for x in 'NEVER SURRENDER'))
RAND = np.random.get_state()

TARGET_ = 'Страховые выплаты'

In [4]:
df_insurance = pd.read_csv('/datasets/insurance.csv')
#df_insurance.profile_report()

Основные выводы по профайлингу:
- в датасете есть дубликаты
- пропущенных значений нет
- аномальных значений нет
- целевой признак распределен неравномерно, большинство людей (**88.7%**) страховых выплат не получали
- можно выделить категориальные признаки (пол) - может принимать всего 2 значения, 0 и 1 => можно не делать OHE
- большая корреляция между возрастом и страховыми выплатами

В таблице есть признаки с типом float. Допустим, что наших данных значимыми являются значения до 3 знаков после запятой, проверим, есть ли числа со значимой дробной частью:

In [6]:
(round(df_insurance, 3) % 1 != 0).sum()

Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

Дробных значений нет выбранной значимости нет, переведем все в int

In [7]:
df_insurance = df_insurance.astype('int32')

Т.к. нет уникального номера клиента можно допустить, что дубликаты - разные люди. 

Переведем названия столбцов в snake_case, т.к. планируется защита данных - можно столбцам дать обезличенные названия и сохранить словарь соответствий.

In [8]:
new_names = [f'feature_{i+1}'  if itm != TARGET_ else 'target' for i,itm in enumerate(df_insurance.columns)]
dict_columns = dict(zip(df_insurance.columns, new_names))
df_insurance.columns = new_names
dict_columns

{'Пол': 'feature_1',
 'Возраст': 'feature_2',
 'Зарплата': 'feature_3',
 'Члены семьи': 'feature_4',
 'Страховые выплаты': 'target'}

Разобьем датасет на целевой признак и признаки:

In [9]:
df_target = df_insurance['target']
df_features = df_insurance.drop(columns='target')

_______________________
**Вывод:** данные были загружены, изучены и обработаны. Столбцы переименованы. Датасет разбит на целевой признак и признаки.

## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$



_________________________
**Необходимо проверить:**  

Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?
_________________________
**Проверка:** 

***Используемые формулы (свойства обратимых матриц)***
$$
X(X)^{-1} = E
$$
$$
XE = X
$$
$$
(XY)^{-1} = (Y)^{-1}(X)^{-1}
$$
$$
(XY)^T = (Y)^T(X)^T
$$


В формулу предсказаний подставим формулу обучения:
$$
a = Xw = X(X^T X)^{-1} X^T y
$$



И домножим матрицу признаков на некую матрицу P 


Матрица X - матрица признаков, в нашем случае имеет 4 признака. Количество строк может быть любое и не должно влиять на возможность решения, для примера возьмем 50. Размер матрицы 50x4 <br>
Таким образом, для корректного умножения матриц и результата - P должна иметь размеры 4x4


$$
a_p = X_{50x4}P_{4x4}( (X_{50x4}P_{4x4})^T (X_{50x4}P_{4x4}))^{-1} (X_{50x4}P_{4x4})^T y  
$$


раскроем скобки части:


$$
( (X_{50x4} P_{4x4})^T (X_{50x4} P_{4x4}))^{-1} =  (P_{4x4}^T X_{4x50}^T X_{50x4}P_{4x4})^{-1} = (P_{4x4}^T (X_{4x50}^T X_{50x4})_{4x4}P_{4x4})^{-1} =
P_{4x4}^{-1}(X_{4x50}^T X_{50x4})^{-1}(P_{4x4}^T)^{-1}
$$

и подставим
$$
a_p = X_{50x4}P_{4x4} P_{4x4}^{-1}(X_{4x50}^T X_{50x4})^{-1}(P_{4x4}^T)^{-1} P_{4x4}^T X_{4x50}^T y  
$$

сократим P
$$
a_p =  X_{50x4} (X_{4x50}^T X_{50x4})^{-1} X_{4x50}^T y  
$$

Видно, что P сократилось полностью и формула $a_p$ полностью аналогична формуле a. <br>    
         
Значит, домножение матрицы признаков на некую матрицу P не влияет на предсказания.
    
    
_________________________
**Вывод:**

При домножение мартицы признаков на некую обратимую матрицу P не влияет на качество линейной регрессии, т.к. полученные предсказанные значения аналогичны предсказанным значениям, полученным без домножения.

## Алгоритм преобразования

**Алгоритм**

Имеющуюся матрицу признаков размером (x,y) домножим на обратимую матрицу размера (y,y)

**Обоснование**

Ранее было выявлено, что домножение матрицы на обратимую матрицу не влияет на результат предсказания. Так же


$$
A_(n,m) B_(m,k) = C_(n,k)
$$

т.е. для того, чтоб полученная матрица C совпадала размерностью с матрицей A должно k=m


Так же эта формула не зависит от количества строк  и может шифровать данные любого размера без изменения ключа.

Данные содержат признаки разных "типов" и порядка, например пол (0 и 1), возраст (от 18 до 65), зарплата (от 5300 до 79000), которые могут косвенно дать представление о самом признаке.

Каждый элемент произведения матриц рассчитывается по формуле
$$
C_(i,j) = \sum_{k=1}^n A_(i,k) B_(k,j)
$$
Получается, что в формировании каждого элемента новой матрицы участвуют все признаки(одна строка), что должно убрать разницу в типах и порядках, для примера:

$$
\begin{pmatrix} 1&41&49600\\ 0&46&38000 \\ 0 &29& 	21000 \\0&21&41700	  \end{pmatrix}\begin{pmatrix} 5&2&3\\ 4&5&6 \\ 2&3&5 	  \end{pmatrix} =\begin{pmatrix} 99369& 149007& 248249 \\
       76184& 114230& 190276 \\
        42116&  63145& 105174 \\
       83484& 125205& 208626\end{pmatrix}
$$

Можно заметить, что все признаки потеряли свою уникальность и по полученным данным нет возможности получить информацию о том, какого "типа" и порядка был признак до изменений.

Ранее было определено, что
$$
w_p = P^{-1}(X^T X)^{-1} X^T y  = P^{-1}w
$$

получается, что получившийся для новой матрицы вектор весов линейной регрессии уже учитывает изменение самой матрицы и расчитан с учетом этих измнений. 

По свойствам матриц, если $AB = C$, то $CB^{-1}=A$, что позволяет проводить дешифрование данных.

___________________________________
**Использумые функции**

In [10]:
class matrix_encryptor:  
    #create key
    def key_generate(self) -> np.ndarray:
        div_matrix = np.random.randint(*self.interval, (self.n, self.n)) 
        return div_matrix if np.linalg.det(div_matrix) != 0 else self.key_generate()
    
    #encrypt data
    def encrypt(self) -> None:
        self.data_encrypt = self.data @ self.key
        self.data_encrypt.columns = self.col_names
        
        
    def __init__(self, 
                 data: pd.DataFrame, 
                 interval: tuple = (2,10),
                 col_names: list = new_names[:-1],
                 rnd: int = RAND) -> None:
        
        self.data = data
        self.col_names = col_names
        self.n = self.data.shape[1]
        self.interval = interval
        np.random.set_state(rnd)
        self.key =  self.key_generate()
        self.encrypt()
        
    #decrypt data and check 
    def decrypt(self, show_result: bool = True) -> None:
        self.data_decrypt = round(self.data_encrypt  @ np.linalg.inv(self.key)).astype(int)
        self.data_decrypt.columns = self.col_names
        if show_result:
            display(self.data_decrypt.head())
            print(f'\n\nDifferent\n{(self.data_decrypt != self.data).sum()}')
    

_________________________________________
Создадим вспомогательную матрицу P с учетом критерия обратимости 

**Критерий обратимости**: матрица обратима тогда и только тогда, когда она невырождена, то есть её определитель не равен нулю

In [11]:
data_insurance = matrix_encryptor(df_features)
data_insurance.key

array([[3, 9, 9, 5],
       [6, 3, 4, 9],
       [8, 2, 3, 4],
       [2, 7, 3, 4]])

В классе было произведено домножение признаков на матрицу ключа, посмотрим результат:

In [12]:
data_insurance.data_encrypt.head()

,feature_1,feature_2,feature_3,feature_4
0,397051,99339,148976,198778
1,304278,76145,114187,152418
2,168174,42087,63116,84261
3,333730,83477,125190,166997
4,208971,52293,78421,104657


Полученные значения признаков действительно не дают представления о том, какие значения были без защиты.

______________________
Так же стоит проверить, что мы сможем получить исходные данные из зашифрованных:

In [13]:
data_insurance.decrypt()

,feature_1,feature_2,feature_3,feature_4
0,1,41,49600,1
1,0,46,38000,1
2,0,29,21000,0
3,0,21,41700,2
4,1,28,26100,0




Different
feature_1    0
feature_2    0
feature_3    0
feature_4    0
dtype: int64


Отличающихся значений нет. Значит, полученная матрица зашифрованных признаков корректна.

**Вывод:**<br>
Был разработан алгоритм шифрования, создан и проверен на корректность "ключ". 


Для дальнейшей работы были зашифрованы данные и сделан вывод, что шифрование не дает возможности определить тип признака и его значение. Так же было проверено, что возможно дешифрование данных.


## Проверка алгоритма

_____________________
**Используемые функции**


In [14]:
def create_model(features: pd.DataFrame, target: pd.Series = df_target, name: str = '',  **kwargs):
    model = LinearRegression(**kwargs)
    all_scores = np.round(cross_val_score(model, features, target, scoring='r2', cv=5), 5)
    return [name, all_scores, all_scores.mean()]

____________________________________

Т.к. будет использоваться кросс-валидация нет необходимости разбивать выборку на тестовую и валидационные части.

In [15]:
result = []

In [16]:
result.append(create_model(data_insurance.data, name='Before encryption'))

In [17]:
result.append(create_model(data_insurance.data_encrypt, name='After encryption'))

In [18]:
pd.DataFrame(result, columns=['data', 'R2_scores', 'R2_score_mean'])

,data,R2_scores,R2_score_mean
0,Before encryption,"[0.40104, 0.44663, 0.41583, 0.41471, 0.43736]",0.423114
1,After encryption,"[0.40104, 0.44663, 0.41583, 0.41471, 0.43736]",0.423114


В обоих случаях метрики совпадают. Значит, алгоритм шифрования работает верно и не влияет на качество модели.

## Вывод

В ходе работы был предложен алгоритм шифрования, основанный на произведении матриц.

Было показано:
- математическое обоснование алгоритма
- полученные после шифрования признаки не дают представления о составе и значениях первоначальных признаков
- зашифрованные данные можно расшифровать при помощи "ключа"
- метрики у зашифрованных и изначальных данных не отличаются


Таким образом, разработанный алгоритм можно рекомендовать для шифрования данных клиентов.